In [3]:
from django.contrib.postgres.aggregates import StringAgg
from django_cte import CTEQuerySet
from django_cte import With 
from django.db.models import CharField
from django.db.models import OuterRef
from django.db.models import Subquery
from django.db.models import Value
from django.db.models import When
from django.db.models.functions.comparison import Cast
from django.db.models.functions import Coalesce
from django.db.models.functions import Concat
from django.db.models.functions.comparison import Cast
from django.db.models.functions.comparison import NullIf
from django.db.models.functions.text import Trim

from common.models.utils import override_current_transaction


workbasket_pk = 349
measure_sid = 20185730

workbasket = WorkBasket.objects.get(pk=workbasket_pk)
current_transaction = workbasket.current_transaction


def get_measure_queryset(current_transaction, sid):
    # Counterintuitively, TrackedModelQuerySet.get_latest_version() fails
    # to get the latest (or any) version of a Model instance that is not in
    # an approved WorkBasket - i.e. its status is in appoved_status.
    # This is related to VersionGroup.current_version not being set.
    # VersionGroup.current_version is only set in TrackedModel.save() when the
    # containing WorkBasket.status is one of WorkflowStatus.approved_statuses()).
    # Therefore, TrackedModelQuerySet.approved_up_to_transaction() is the way
    # it's done in Tamato's Views and Forms.
    m_qs = (
        Measure.objects.filter(sid=sid)
            .approved_up_to_transaction(
                current_transaction
            )
    )
    return m_qs


def with_duty_sentence(current_transaction, measure_qs):
    # Get the version groups of those measures that components must be
    # joined to - this narrows to a relevant, more manageable components
    # queryset for CTE construction.
    measure_version_group_ids = set(
        measure_qs.values_list("version_group_id", flat=True)
    )
    
    components_qs = MeasureComponent.objects.filter(
        component_measure__version_group__id__in=measure_version_group_ids
    ).approved_up_to_transaction(
        # Is transaction filtering really the right approach when some other
        # attribute could equally well have been applied to filter measure_qs?
        current_transaction
    ).annotate(
        measure_version_group_id=F("component_measure__version_group_id")
    ).values(
        "transaction",
        "measure_version_group_id",
    ).annotate(
        duty_sentence=StringAgg(
            expression=Trim(
                Concat(
                    Case(
                        When(
                            Q(duty_expression__prefix__isnull=True) |
                            Q(duty_expression__prefix=""),
                            then=Value(""),
                        ),
                        default=Concat(
                            F("duty_expression__prefix"),
                            Value(" "),
                        ),
                    ),
                    "duty_amount",
                    Case(
                        When(
                            monetary_unit=None,
                            duty_amount__isnull=False,
                            then=Value("%"),
                        ),
                        When(
                            duty_amount__isnull=True,
                            then=Value(""),
                        ),
                        default=Concat(
                            Value(" "),
                            F("monetary_unit__code"),
                        ),
                    ),
                    # TODO: Add remaining cases.
                    output_field=CharField(),
                ),
            ),
            delimiter=" ",
            ordering="duty_expression__sid",
        )
    )
    
    print(f"*** components_qs = {components_qs}")
    print(f"*** components_qs.count() = {components_qs.count()}")
    
    cte = With(components_qs)

    joined_measure_qs = (
        cte.join(
            measure_qs,
            version_group_id=cte.col.measure_version_group_id
        ).with_cte(
            cte
        ).annotate(
            duty_sentence=cte.col.duty_sentence
        )
    )
    
    return joined_measure_qs


measure_qs = get_measure_queryset(current_transaction, measure_sid)
print(f"*** measure_qs = {measure_qs}")
print(f"*** measure_qs.count() = {measure_qs.count()}")

measure_qs = with_duty_sentence(current_transaction, measure_qs)
for m in measure_qs:
    print("Measure:")
    print(f"    sid:{m.sid}")
    print(f"    duty_sentence={m.duty_sentence}")


*** measure_qs = <MeasuresQuerySet [<Measure: 20185730>]>
*** measure_qs.count() = 1
*** components_qs = <TrackedModelQuerySet [{'transaction': 5118158, 'measure_version_group_id': 9931455, 'duty_sentence': '4.000%'}]>
*** components_qs.count() = 1
Measure:
    sid:20185730
    duty_sentence=4.000%
